In [1]:
import os
import json
from termcolor import colored
from llmagent import LLMAgent

In [2]:
class IntentFormalizer(LLMAgent):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def generate_task(self, instruction):
        system_prompt = f'''
You are a helpful assistant.
The user needs to talk to a customer support agent to solve their issue.
However, their request in very complicated.
Your job is to extract the different tasks from their request into a list so that it is easier for the support agent to understand and solve it. 
Out your response in json in the following format:
{{
    task: <tasks_list>
}}
'''
        messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": instruction}]
        res = self.llm_client.complete(model = self.model_name, messages=messages, response_format="json_object").choices[0].message['content']
        res = json.loads(res)
        return res['task']

In [3]:
def get_tasks(task_split, env_type):
    filename = f'one_shot_{env_type}_{task_split}.json'
    tasks = json.load(open(filename))
    return tasks

In [11]:
def formalize_tasks(task_split, env_type):
    output_file = f'formalized_tasks_{env_type}_{task_split}.json'
    tasks = get_tasks(task_split, env_type)
    intent_formalizer = IntentFormalizer()
    formalized_tasks = []
    for i in range(len(tasks)):
        print(i)
        print(colored(tasks[i], 'blue'))
        formalized_task = intent_formalizer.generate_task(tasks[i])
        print(colored(formalized_task, 'green'))
        formalized_tasks.append(formalized_task)
    with open(output_file, 'w') as f:
        json.dump(formalized_tasks, f)

In [13]:
formalize_tasks('train', 'retail')

0
I would like to return order #W6067464, which contains the following items: Electric Kettle and Wall Clock. Please process the refund to my credit card ending in 4190576. My name is Omar Anderson and my zip code is 19031.
['Initiate return process for order #W6067464.', 'Process refund for both Electric Kettle and Wall Clock.', 'Ensure refund is issued to credit card ending in 4190576.', 'Verify customer information: Name - Omar Anderson, Zip code - 19031.']
1
I would like to return order #W6619432 via PayPal account paypal_3738584. The items to be returned are: Dumbbell Set and Yoga Mat. My name is Sophia Nguyen and my zip code is 20171.
['Initiate a return for order #W6619432.', 'Process the return via PayPal account paypal_3738584.', 'Return the following items: Dumbbell Set and Yoga Mat.', 'Verify the customer’s name as Sophia Nguyen.', 'Verify the customer’s zip code as 20171.']
2
I want to return order #W4435622: Water Bottle using gift card gift_card_1725971. My name is James 